# Principle Component Analysis with Iris Dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set up the environment for using pyspark
import findspark
findspark.init()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.ml.linalg import Vectors

In [ ]:
# Create Application Context
spark = SparkSession.builder.appName("PCA Example").getOrCreate()
sc = spark.sparkContext

In [ ]:
# NOTE The CSV file does not have any headers
sdf = spark.read.csv('datasets/iris.csv').toDF('Sepal_L', 'Sepal_W', 'Petal_L', 'Petal_W', 'Species')

In [ ]:
sdf.show()

In [ ]:
sdf.printSchema()

### Since the feature columns are string convert them to floats

In [ ]:
from pyspark.sql.functions import col
cols = sdf.columns
# ignore Species column
cols = cols[:-1]
for col_name in cols:
    sdf = sdf.withColumn(col_name, col(col_name).cast('float'))

In [ ]:
sdf.printSchema()

### Select Features

In [ ]:
from pyspark.ml.feature import VectorAssembler
vassemb = VectorAssembler(inputCols = cols, outputCol = 'features')
ndf = vassemb.transform(sdf)
ndf = ndf.select(['features'])
ndf.show(3, truncate = False)

### Standardize the features

In [ ]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="stdFeatures",
                            withStd=False, withMean=True)

In [ ]:
scalerModel = scaler.fit(ndf)

In [ ]:
scalerModel = scaler.fit(ndf)

In [ ]:
scaledData = scalerModel.transform(ndf)

In [ ]:
scaledData.select("stdFeatures").show(3, truncate = False)

### Apply Principle Component Analysis (PCA)

In [ ]:
from pyspark.ml.feature import PCA

In [ ]:
pca = PCA(k = 4, inputCol = scaler.getOutputCol(), outputCol = 'pcaFeatures')

In [ ]:
model = pca.fit(scaledData)

In [ ]:
transformed_feature = model.transform(scaledData)

In [ ]:
transformed_feature.select('pcaFeatures').show(3, truncate = False)

In [ ]:
np.round(100.00*model.explainedVariance.toArray(),4)

In [ ]:
pcs = np.round(model.pc.toArray(), 4)
pcs

In [ ]:
df_pc = pd.DataFrame(pcs, columns = ['PC1','PC2','PC3','PC4'], index = cols)
df_pc

In [ ]:
df_pc['PC1']

In [ ]:
transformed_feature.select('pcaFeatures').show(10, truncate = False)